In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# General imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model
import seaborn as sns
from mlxtend.plotting import plot_decision_regions
from sklearn.metrics import confusion_matrix
import cv2
import PIL

In [ ]:
# Set up ImageDataGenerator
train_imagegen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                   zoom_range=[0.6,1],
                                   rotation_range=10,
                                   brightness_range=([0.6, 1.5]),
                                   horizontal_flip=True,
                                   validation_split=0.06) # this will set aside a part of training set for validation data
test_imagegen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                   zoom_range=[0.6,1],
                                   rotation_range=10,
                                   brightness_range=([0.6, 1.5]),
                                   horizontal_flip=True)
# Bring the data in
train_generator = train_imagegen.flow_from_directory(
                                    '../input/brain-tumor-classification-mri/Training',
                                    target_size=(200,200),
                                    batch_size=20,
                                    class_mode='categorical',
                                    subset='training')

test_generator = test_imagegen.flow_from_directory(
                                    '../input/brain-tumor-classification-mri/Testing',
                                    target_size=(200,200),
                                    batch_size=20,
                                    class_mode='categorical')

val_generator = train_imagegen.flow_from_directory(
                                    '../input/brain-tumor-classification-mri/Training',
                                    target_size=(200,200),
                                    batch_size=20,
                                    class_mode='categorical',
                                    subset='validation')

In [ ]:
def visualize_training_results(history):
    '''
    From https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/
    
    Input: keras history object (output from trained model)
    '''
    fig, (ax1, ax2) = plt.subplots(2, sharex=True)
    fig.suptitle('Model Results')

    # summarize history for accuracy
    ax1.plot(history.history['acc'])
    ax1.plot(history.history['val_acc'])
    ax1.set_ylabel('Accuracy')
    ax1.legend(['train', 'test'], loc='upper left')
    # summarize history for loss
    ax2.plot(history.history['loss'])
    ax2.plot(history.history['val_loss'])
    ax2.set_ylabel('Loss')
    ax2.legend(['train', 'test'], loc='upper left')
    
    plt.xlabel('Epoch')
    plt.show()

In [ ]:
# Visualize (code from https://github.com/austint1121/OES-PneumoniaClassification/blob/main/Final_Notebook.ipynb)
train_batch = train_generator.next()
fig, axes = plt.subplots(2, 5, figsize=(16, 8))
    
for i in range(10):
    # Load image into numpy array and re-scale
    img = np.array(train_batch[0][i] * 255, dtype='uint8')
    ax = axes[i // 5, i % 5]
    ax.imshow(img)
fig.suptitle('Training Images')
plt.tight_layout()
plt.show()

## **Getting a look at the Distribution of Different Tumor Types**

In [ ]:
train_generator.mode

In [ ]:
train_generator.class_indices

In [ ]:
train_generator[0].

In [ ]:
train_tumor_types = pd.DataFrame(train_generator.classes)
train_values = train_tumor_types.value_counts()
train_values

In [ ]:
train_tumor_types.rename(columns={0:'Tumor Type'}, inplace=True)

In [ ]:
# Preparations for visualization
no_tumor = len(train_tumor_types[train_tumor_types['Tumor Type'] == 2])
glioma = len(train_tumor_types[train_tumor_types['Tumor Type'] == 0])
meningioma = len(train_tumor_types[train_tumor_types['Tumor Type'] == 1])
pituitary = len(train_tumor_types[train_tumor_types['Tumor Type'] == 3])

In [ ]:

fig, ax = plt.subplots(figsize=(10,8))
ax.bar(x=['No Tumor', 'Glioma', 'Meningioma', 'Pituitary'], height = [no_tumor, glioma, meningioma, pituitary])
ax.set(xlabel='', ylabel='Number of Images', title='Distribution of Brain Tumor Type');


## **Taking a Look at Different Tumor Types**

In [ ]:
# View a Glioma tumor
glioma1 = PIL.Image.open('../input/brain-tumor-classification-mri/Training/glioma_tumor/gg (1).jpg')
glioma1

In [ ]:
glioma1.mode

In [ ]:
glioma1.size

## **First Baseline Model**

In [ ]:
baseline = keras.Sequential()
baseline.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(200,200,3)))
baseline.add(layers.MaxPooling2D(2,2))
baseline.add(layers.Conv2D(64, (3,3), activation='relu'))
baseline.add(layers.MaxPooling2D(2,2))

baseline.add(layers.Flatten())
baseline.add(layers.Dense(128, activation='relu'))
baseline.add(layers.Dense(4, activation='softmax'))

baseline.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])

baseline_results = baseline.fit_generator(train_generator,
                                         steps_per_epoch=2699/20,# number of samples / batch size
                                         epochs=10,
                                         validation_data=test_generator)

In [ ]:
visualize_training_results(baseline_results)

## **Adding Class Weights to Baseline CNN**

In [ ]:
base_weights = keras.Sequential()
base_weights.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(200,200,3)))
base_weights.add(layers.MaxPooling2D(2,2))
base_weights.add(layers.Conv2D(64, (3,3), activation='relu'))
base_weights.add(layers.MaxPooling2D(2,2))

base_weights.add(layers.Flatten())
base_weights.add(layers.Dense(128, activation='relu'))
base_weights.add(layers.Dense(4, activation='softmax'))

base_weights.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])

multi_weights = {0: 1,
                1: 1 ,
                2: 2,
                3: 1,}

base_weights_results = base_weights.fit_generator(train_generator,
                                          class_weight=multi_weights,
                                         steps_per_epoch=2699/20,# number of samples / batch size
                                         epochs=10,
                                         validation_data=test_generator)


## **Adding Batch Normalization**

In [ ]:
norm = keras.Sequential()
norm.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(200,200,3)))
norm.add(layers.BatchNormalization())
norm.add(layers.MaxPooling2D(2,2))
norm.add(layers.Conv2D(64, (3,3), activation='relu'))
norm.add(layers.BatchNormalization())
norm.add(layers.MaxPooling2D(2,2))

norm.add(layers.Flatten())
norm.add(layers.Dense(128, activation='relu'))
norm.add(layers.BatchNormalization())
norm.add(layers.Dense(4, activation='softmax'))

norm.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])

multi_weights = {0: 1,
                1: 1 ,
                2: 2,
                3: 1,}

norm_results = norm.fit_generator(train_generator,
                                          class_weight=multi_weights,
                                         steps_per_epoch=2699/20,# number of samples / batch size
                                         epochs=10,
                                         validation_data=test_generator)

In [ ]:
visualize_training_results(norm_results)

## **Using Pre-Trained VGG-19 Weights**

In [ ]:
from keras.applications.vgg19 import VGG19
cnn_vgg = VGG19(weights='imagenet',
               include_top=False,
               input_shape=(200,200,3))

In [ ]:
# Making early stop for model
early_stop2 = [EarlyStopping(monitor='val_loss', patience=12, restore_best_weights=True),
            ModelCheckpoint(filepath='best_model.h5', monitor='val_loss',
                           save_best_only=True)]

In [ ]:
cnn_vgg.summary()

In [ ]:
# Build first model using pretrained VGG 19 as first layer, and then some dense layers on top
pretrained = keras.Sequential()
pretrained.add(cnn_vgg)
pretrained.add(layers.Flatten())
pretrained.add(layers.Dense(128, activation='relu'))
pretrained.add(layers.Dense(4, activation='softmax'))

In [ ]:
# Make the pretrianed layer untrainable so that during optimization, its weights don't change
cnn_vgg.trainable = False

In [ ]:
# Check to see that the pretrained layer is not trainable but that all others are
for layer in pretrained.layers:
    print(layer.name, layer.trainable)
    
print(len(pretrained.trainable_weights))

In [ ]:

pretrained.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])

pretrained_results = pretrained.fit_generator(train_generator,
                                         steps_per_epoch=2699/20,# number of samples / batch size
                                         epochs=20,
                                        callbacks= early_stop2,
                                         validation_data=test_generator)

In [ ]:
visualize_training_results(pretrained_results)

**Analysis of Model**

Training accuracy is 97% and testing accuracy is 77%, while training loss is 9% and testing loss is 184%. Testing recall is 77%. The model is definitely overfit, there is a lot of loss, and it is important for recall to be higher, so this model requires a lot of tuning. In the next iteration I will see if unfreezing some of the outer layers so that they can train on these specific images will help imrpove the model.

## **Fine Tuning the Outer Layers of the Pretrained Network**

In [ ]:
# Build first model using pretrained VGG 19 as first layer, and then some dense layers on top
b5_c1 = keras.Sequential()
b5_c1.add(cnn_vgg)
b5_c1.add(layers.Flatten())
b5_c1.add(layers.Dense(128, activation='relu'))
b5_c1.add(layers.Dense(4, activation='softmax'))

In [ ]:
# Unfreezing the base
cnn_vgg.trainable = True

In [ ]:
# Re-freezing everything except for the last layer of the pretrained CNN
# Code structure from https://github.com/learn-co-curriculum/dsc-using-pretrained-networks-codealong
cnn_vgg.trainable = True
for layer in  cnn_vgg.layers:
    if layer.name == 'block5_conv1':
        layer.trainable = True
    else:
        layer.trainable = False
        

In [ ]:
# Checking to see that only the 'block5_conv1' layer is unfrozen
for layer in cnn_vgg.layers:
    print(layer.name, layer.trainable)
    
print(len(cnn_vgg.trainable_weights))

In [ ]:
b5_c1.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])

b5_c1_results = pretrained.fit_generator(train_generator,
                                         steps_per_epoch=2699/20,# number of samples / batch size
                                         epochs=20,
                                        callbacks= early_stop2,
                                         validation_data=test_generator)

In [ ]:
visualize_training_results(b5_c1_results)

**Analysis of Model**

The best epoch had a training accuracy of ~ 96% and a testing accuracy of ~ 76%, with a training loss of ~ 12% and a testing loss of ~ 155%. It has a testing recall of 75%. These results are very similar to the previous model, so it looks like unfreezing a layer did not improve the model much. Will see if unfreezing one more layer will help.

### **Analysis of Model**



## **Unfreezing Two layers from Pretrained Nework (VGG)**

In [ ]:
# Build first model using pretrained VGG 19 as first layer, and then some dense layers on top
b5_c1c2 = keras.Sequential()
b5_c1c2.add(cnn_vgg)
b5_c1c2.add(layers.Flatten())
b5_c1c2.add(layers.Dense(128, activation='relu'))
b5_c1c2.add(layers.Dense(4, activation='softmax'))

In [ ]:
# Re-freezing everything except for the last layer of the pretrained CNN
# Code structure from https://github.com/learn-co-curriculum/dsc-using-pretrained-networks-codealong
cnn_vgg.trainable = True
for layer in  cnn_vgg.layers:
    if (layer.name == 'block5_conv1') | (layer.name == 'block5_conv2'):
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
# Checking to see that only 'block5_conv1' and 'block5_conv3' layers are unfrozen
for layer in cnn_vgg.layers:
    print(layer.name, layer.trainable)
    
print(len(cnn_vgg.trainable_weights))

In [ ]:
b5_c1c2.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])

b5_c1c2_results = pretrained.fit_generator(train_generator,
                                         steps_per_epoch=2699/20,# number of samples / batch size
                                         epochs=20,
                                        callbacks= early_stop2,
                                         validation_data=test_generator)

In [ ]:
visualize_training_results(b5_c1c2_results)

**Analysis of Model**

The epoch with the lowest loss has a training accuracy of 93% and a testing accuracy of 76%, with a training loss of 18% and a testing loss of 118%. Testing recall is 69%. It looks like unfreezing two layers has helped decrease the testing a loss a bit; without any unfrozen layers, testing loss is around 184%. 

## **Unfreezing all Four Convolutional Layers in Block 5 of Pretrained Neural Network**

In [ ]:
# Build first model using pretrained VGG 19 as first layer, and then some dense layers on top
b5_c1c2c3c4 = keras.Sequential()
b5_c1c2c3c4.add(cnn_vgg)
b5_c1c2c3c4.add(layers.Flatten())
b5_c1c2c3c4.add(layers.Dense(128, activation='relu'))
b5_c1c2c3c4.add(layers.Dense(4, activation='softmax'))

In [ ]:
# Re-freezing everything except for the last layer of the pretrained CNN
# Code structure from https://github.com/learn-co-curriculum/dsc-using-pretrained-networks-codealong
cnn_vgg.trainable = True
for layer in  cnn_vgg.layers:
    if (layer.name == 'block5_conv1') | (layer.name == 'block5_conv2') | (layer.name == 'block5_conv3') | (layer.name == 'block5_conv4'):
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
# Checking to see that only 'block5_conv1' and 'block5_conv3' layers are unfrozen
for layer in cnn_vgg.layers:
    print(layer.name, layer.trainable)
    
print(len(cnn_vgg.trainable_weights))

In [ ]:
b5_c1c2c3c4.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])

b5_c1c2c3c4_results = pretrained.fit_generator(train_generator,
                                         steps_per_epoch=2699/20,# number of samples / batch size
                                         epochs=20,
                                        callbacks= early_stop2,
                                         validation_data=test_generator)

**Analysis of Model**

The epoch with the lowest loss has a training accuracy of 97% and a testing accuracy of 77%, with a training loss of 8% and a testing loss of 173%. Testing recall is 76%. Testing loss seems to have increased from the last model iteration which only had two layers unfrozen, so I will return to just unfreezing two layers in the iteration, and will also try seeing if batch normalization and dropout layers improve the model.

## **Adding Batch Normalization and Dropout layers to Model with Two Unfrozen Layers**

In [ ]:
# Build first model using pretrained VGG 19 as first layer, and then some dense layers on top
n_b5_c1c2 = keras.Sequential()
n_b5_c1c2.add(cnn_vgg)
n_b5_c1c2.add(layers.Dropout(0.25))
n_b5_c1c2.add(layers.BatchNormalization())

n_b5_c1c2.add(layers.Flatten())
n_b5_c1c2.add(layers.Dense(128, activation='relu'))
n_b5_c1c2.add(layers.Dropout(0.3))
n_b5_c1c2.add(layers.BatchNormalization())
n_b5_c1c2.add(layers.Dense(4, activation='softmax'))

In [ ]:
# Re-freezing everything except for the last layer of the pretrained CNN
# Code structure from https://github.com/learn-co-curriculum/dsc-using-pretrained-networks-codealong
cnn_vgg.trainable = True
for layer in  cnn_vgg.layers:
    if (layer.name == 'block5_conv1') | (layer.name == 'block5_conv2'):
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
# Checking to see that only 'block5_conv1' and 'block5_conv3' layers are unfrozen
for layer in cnn_vgg.layers:
    print(layer.name, layer.trainable)
    
print(len(cnn_vgg.trainable_weights))

In [ ]:
n_b5_c1c2.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])

n_b5_c1c2_results = pretrained.fit_generator(train_generator,
                                         steps_per_epoch=2699/20,# number of samples / batch size
                                         epochs=20,
                                        callbacks= early_stop2,
                                         validation_data=test_generator)

In [ ]:
visualize_training_results(n_b5_c1c2_results)

**Analysis of Model**

The epoch with the lowest testing loss had a training accuracy of around 97% and a testing accuracy of 78%, with a training loss of 7% and a testing loss of 151%. Testing recall is 77%. Testing loss is slightly better than in the last model, but overall it is not much better. Maybe a different pre-trained network would be better.

## **Using Pretrained Weights from VGG-16**

In [ ]:
from keras.applications.vgg16 import VGG16
vgg16 = VGG16(weights='imagenet',
               include_top=False,
               input_shape=(200,200,3))

In [ ]:
vgg16.summary()

In [ ]:
# Build first model using pretrained VGG 19 as first layer, and then some dense layers on top
vgg16_model = keras.Sequential()
vgg16_model.add(vgg16)
vgg16_model.add(layers.Flatten())
vgg16_model.add(layers.Dense(128, activation='relu'))
vgg16_model.add(layers.Dense(4, activation='softmax'))

In [ ]:
vgg16.trainable = False

In [ ]:
# Check to see that the pretrained layer is not trainable but that all others are
for layer in vgg16_model.layers:
    print(layer.name, layer.trainable)
    
print(len(vgg16_model.trainable_weights))

In [ ]:
vgg16_model.compile(optimizer='adam',
                    loss='categorical_crossentropy',
                    metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])



In [ ]:
vgg16_model_results = vgg16_model.fit_generator(train_generator,
                                         steps_per_epoch=2699/20,# number of samples / batch size
                                         epochs=20,
                                        callbacks= early_stop2,
                                         validation_data=test_generator)

**Analysis of Model**

The epoch with the lowest testing loss had a training accuracy of 86% and a testing accuracy of 68%, with a training loss of 37% and a testing loss of 110%. Testing recall is 65%. Testing loss is slightly lower than the previous model using VGG19, but accuracy and recall is significantly lower, so it looks like using the VGG network with more layers is better. However, it is still a good idea to continue to try different pretrained networks.

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
incep = InceptionV3(weights='imagenet',
               include_top=False,
               input_shape=(200,200,3))

In [ ]:
incep.summary()


In [ ]:
# Build first model using pretrained VGG 19 as first layer, and then some dense layers on top
incep_model = keras.Sequential()
incep_model.add(incep)
incep_model.add(layers.Flatten())
incep_model.add(layers.Dense(512, activation='relu'))
incep_model.add(layers.Dense(4, activation='softmax'))

In [ ]:
incep.trainable = False

In [ ]:
# Check to see that the pretrained layer is not trainable but that all others are
for layer in incep_model.layers:
    print(layer.name, layer.trainable)
    
print(len(incep_model.trainable_weights))

In [ ]:
incep_model.compile(optimizer='adam',
                    loss='categorical_crossentropy',
                    metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])

incep_model_results = incep_model.fit_generator(train_generator,
                                         steps_per_epoch=2699/20,# number of samples / batch size
                                         epochs=20,
                                        callbacks= early_stop2,
                                         validation_data=test_generator)

In [ ]:
visualize_training_results(incep_model_results)

**Analysis of Model**

The epoch with the lowest testing loss has a training accuracy of 91% and a testing accuracy of 75%, with a training loss of 25% and a testing loss of 90%. Testing recall is around 73% This pretrained network resulted in the lowest testing loss yet, which is very promising; in the next few iterations I will try fine tuning this inception pretrained network.

## **Fine Tuning of Inception Network**
Adding class weights to account for class imbalance, adding batch normalization and dropout layer, unfreezing outer most convolution layer of Inception Network

In [ ]:
multi_weights = {0: 1,
                1: 1 ,
                2: 2,
                3: 1,}
# Add class weights
# unfreeze outer layer
# decrease learning rate


In [ ]:
# Build first model using pretrained VGG 19 as first layer, and then some dense layers on top
incep2_model = keras.Sequential()
incep2_model.add(incep)
incep2_model.add(layers.Flatten())
incep2_model.add(layers.Dense(512, activation='relu'))
incep2_model.add(layers.BatchNormalization())
incep2_model.add(layers.Dropout(0.3))
incep2_model.add(layers.Dense(4, activation='softmax'))

In [ ]:
incep.trainable = False


In [ ]:
# Check to see that the pretrained layer is not trainable but that all others are
for layer in incep2_model.layers:
    print(layer.name, layer.trainable)
    
print(len(incep2_model.trainable_weights))

In [ ]:
# Re-freezing everything except for the last layer of the pretrained CNN
# Code structure from https://github.com/learn-co-curriculum/dsc-using-pretrained-networks-codealong
incep.trainable = True
for layer in  incep.layers:
    if (layer.name == 'conv2d_93'):
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
for layer in incep.layers:
    print(layer.name, layer.trainable)
    
print(len(incep.trainable_weights))

In [ ]:
multi_weights = {0: 1,
                1: 1 ,
                2: 2,
                3: 1,}

incep2_model.compile(optimizer='adam',
                    loss='categorical_crossentropy',
                    metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])

incep2_model_results = incep2_model.fit_generator(train_generator,
                                        class_weight= multi_weights,
                                         steps_per_epoch=2699/20,# number of samples / batch size
                                         epochs=20,
                                        callbacks= early_stop2,
                                         validation_data=test_generator)

In [ ]:
visualize_training_results(incep2_model_results)

**Analysis of Model**

The epoch with the lowest testing loss had a training accuracy of 94% and a testing accuracy of 77%, with a training loss of 16% and a testing loss of 105%. Testing recall is 69%. Accuracy between this iteration and the last is similar, but testing loss is about ten percentage points higher, so the model still requires some tuning. 

## **Model with Inception Pretrained Network using a smaller learning rate**

In [ ]:
# Build first model using pretrained VGG 19 as first layer, and then some dense layers on top
incep3_model = keras.Sequential()
incep3_model.add(incep)
incep3_model.add(layers.Flatten())
incep3_model.add(layers.Dense(512, activation='relu'))
incep3_model.add(layers.BatchNormalization())
incep3_model.add(layers.Dropout(0.3))
incep3_model.add(layers.Dense(4, activation='softmax'))

In [ ]:
incep.trainable = False

In [ ]:
# Check to see that the pretrained layer is not trainable but that all others are
for layer in incep3_model.layers:
    print(layer.name, layer.trainable)
    
print(len(incep3_model.trainable_weights))

In [ ]:
# Re-freezing everything except for the last layer of the pretrained CNN
# Code structure from https://github.com/learn-co-curriculum/dsc-using-pretrained-networks-codealong
incep.trainable = True
for layer in  incep.layers:
    if (layer.name == 'conv2d_93'):
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
for layer in incep.layers:
    print(layer.name, layer.trainable)
    
print(len(incep.trainable_weights))

In [ ]:
multi_weights = {0: 1,
                1: 1 ,
                2: 2,
                3: 1,}

small_lr_adam = keras.optimizers.Adam(learning_rate=0.0001)

incep3_model.compile(optimizer= small_lr_adam,
                    loss='categorical_crossentropy',
                    metrics=['acc', 'Recall', 'Precision', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])

incep3_model_results = incep3_model.fit_generator(train_generator,
                                        class_weight= multi_weights,
                                         steps_per_epoch=2699/20,# number of samples / batch size
                                         epochs=20,
                                        callbacks= early_stop2,
                                         validation_data=test_generator)

In [ ]:
visualize_training_results(incep3_model_results)

**Analysis of Model**

The epoch with the lowest testing loss had a training accuracy of 94% and a testing acuracy of 76%, with a training loss of 17% and a testing loss of around 99%; the model is still overfitting. Tetsing recall is 71%. Other than the testing loss being slightly lower in this iteration than the last, the numbers between this iteration and the ast are fairly similar; it is possible this model could improve with more training epochs. It is also possible that unfreezing more layers from the pretrained base might help the network better learn and adjust to these specific images.